<a href="https://colab.research.google.com/github/sakur024/emotion-speech-recognition-system/blob/main/SpeechTransformer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Datasets/audio_speech_actors_01-24.zip" -d /content/ravdess

Archive:  /content/drive/MyDrive/Datasets/audio_speech_actors_01-24.zip
   creating: /content/ravdess/audio_speech_actors_01-24/Actor_01/
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav  
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-02-01.wav  
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-01-01-02-01-01.wav  
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-01-01-02-02-01.wav  
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-02-01-01-01-01.wav  
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-02-01-01-02-01.wav  
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-02-01-02-01-01.wav  
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-02-01-02-02-01.wav  
  inflating: /content/ravdess/audio_speech_actors_01-24/Actor_01/03-01-02-02-01-01-01.wav  
  inflating: /content/ravdess/audi

In [ ]:
!pip install transformers datasets torchaudio librosa evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 73.8 MB/s eta 0:00:00


In [ ]:
import os
import torch
import librosa
import numpy as np
import pandas as pd

from datasets import Dataset
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForSequenceClassification,
    TrainingArguments,
    Trainer
)

In [ ]:
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

In [ ]:
import os
import pandas as pd

data_path = "/content/ravdess/audio_speech_actors_01-24"

filepaths = []
emotions = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith(".wav"):

            parts = file.split("-")
            emotion_code = parts[2]   # ✅ THIRD number = emotion

            filepaths.append(os.path.join(root, file))
            emotions.append(emotion_map[emotion_code])

df = pd.DataFrame({
    "path": filepaths,
    "emotion": emotions
})

df.head()

,path,emotion
0,/content/ravdess/audio_speech_actors_01-24/Act...,fearful
1,/content/ravdess/audio_speech_actors_01-24/Act...,surprised
2,/content/ravdess/audio_speech_actors_01-24/Act...,happy
3,/content/ravdess/audio_speech_actors_01-24/Act...,calm
4,/content/ravdess/audio_speech_actors_01-24/Act...,angry


In [ ]:
label_list = sorted(df["emotion"].unique())

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

df["label"] = df["emotion"].map(label2id)

print(label2id)
df.head()

{'angry': 0, 'calm': 1, 'disgust': 2, 'fearful': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprised': 7}


,path,emotion,label
0,/content/ravdess/audio_speech_actors_01-24/Act...,fearful,3
1,/content/ravdess/audio_speech_actors_01-24/Act...,surprised,7
2,/content/ravdess/audio_speech_actors_01-24/Act...,happy,4
3,/content/ravdess/audio_speech_actors_01-24/Act...,calm,1
4,/content/ravdess/audio_speech_actors_01-24/Act...,angry,0


In [ ]:
print("Total samples:", len(df))
print("\nClass distribution:")
print(df["emotion"].value_counts())

Total samples: 1440

Class distribution:
emotion
fearful      192
surprised    192
happy        192
calm         192
angry        192
sad          192
disgust      192
neutral       96
Name: count, dtype: int64


In [ ]:
from transformers import AutoFeatureExtractor, Wav2Vec2ForSequenceClassification

model_name = "facebook/wav2vec2-base"

feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label
)

Loading weights:   0%|          | 0/211 [00:00<?, ?it/s]

Wav2Vec2ForSequenceClassification LOAD REPORT from: facebook/wav2vec2-base
Key                          | Status     | 
-----------------------------+------------+-
quantizer.weight_proj.bias   | UNEXPECTED | 
project_hid.bias             | UNEXPECTED | 
quantizer.weight_proj.weight | UNEXPECTED | 
quantizer.codevectors        | UNEXPECTED | 
project_q.weight             | UNEXPECTED | 
project_q.bias               | UNEXPECTED | 
project_hid.weight           | UNEXPECTED | 
projector.weight             | MISSING    | 
classifier.bias              | MISSING    | 
projector.bias               | MISSING    | 
classifier.weight            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
def preprocess_function(example):
    speech, sr = librosa.load(example["path"], sr=16000)

    inputs = feature_extractor(
        speech,
        sampling_rate=16000,
        padding="max_length",
        truncation=True,
        max_length=16000 * 5,
    )

    return {
        "input_values": inputs["input_values"][0],  # remove batch dim
        "labels": example["label"]
    }

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["path", "label"]])
dataset = dataset.train_test_split(test_size=0.2, seed=42)

dataset = dataset.map(preprocess_function, remove_columns=["path"])

dataset.set_format(type="torch", columns=["input_values", "labels"])

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

In [ ]:
model.freeze_feature_encoder()

In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./emotion_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    fp16=True
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [ ]:
# Freeze CNN feature extractor
model.freeze_feature_encoder()

# Unfreeze last 2 transformer encoder layers
for name, param in model.named_parameters():
    if "encoder.layers.10" in name or "encoder.layers.11" in name:
        param.requires_grad = True

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.188822,0.673611
2,No log,1.111628,0.711806
3,No log,1.025263,0.732639
4,1.066836,0.910938,0.767361
5,1.066836,0.814679,0.802083
6,1.066836,0.702892,0.833333
7,0.654732,0.675284,0.826389
8,0.654732,0.673483,0.826389
9,0.654732,0.643652,0.840278
10,0.654732,0.634990,0.840278


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1440, training_loss=0.726809310913086, metrics={'train_runtime': 432.3657, 'train_samples_per_second': 26.644, 'train_steps_per_second': 3.331, 'total_flos': 5.229377667072e+17, 'train_loss': 0.726809310913086, 'epoch': 10.0})

In [ ]:
save_path = "/content/emotion_wav2vec2_model"

model.save_pretrained(save_path)
feature_extractor.save_pretrained(save_path)

print("Model saved to:", save_path)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Model saved to: /content/emotion_wav2vec2_model


In [ ]:
import shutil

shutil.make_archive("/content/emotion_wav2vec2_model", 'zip', save_path)

print("Zipped successfully")

Zipped successfully


In [ ]:
from google.colab import files

files.download("/content/emotion_wav2vec2_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>